In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical
import parallel
import torch
import movable_wall_parallel
import torch.nn as nn
import numpy as np
from torch.distributions import Categorical
import wandb
RANDOM = 42

import torch.nn.functional as F

In [16]:


############################ HIGHLY IMPORTANT VARIABLES TO SET######################################
GRID_SIZE = 7
NUM_THINGS = 5 # number of things in the grid wall, pred1, pred2, h1, h2, movablewall


INITIALIZATIONS = [
    #'./models/PRED_GROUP_POLICY_BOTH_INDIV_RWD_2700.ckpt', # pred_policy
    #'./models/PRED_GROUP_POLICY_BOTH_INDIV_RWD_2700.ckpt',    # hider_policy
    #'./models/HIDER_GROUP_POLICY_BOTH_INDIV_RWD_1800.ckpt',
    #'./models/HIDER_GROUP_POLICY_BOTH_INDIV_RWD_1800.ckpt',
    RANDOM,
    RANDOM,
    RANDOM,
    RANDOM,
    ]    # hider_2
#should be either RANDOM ; or a path to a pretrained checkpoint (a String)

IS_POLICY_TRAINING = {"preds":True, "hiders":True} #preds, hiders

IS_TRAINING =   [
    IS_POLICY_TRAINING["preds"], #pred1
    IS_POLICY_TRAINING["preds"], #pred2
    IS_POLICY_TRAINING["hiders"], #hider1
    IS_POLICY_TRAINING["hiders"] #hider2
]
#either True or False, if False, weights are frozen (or if random it will stay random)


envname = 'mparallel-walls' #just for wandb logging
CUSTOMENV = movable_wall_parallel.parallel_env(grid_size=GRID_SIZE,walls=True)
# change architecture if needed

ent_coef = 0.4
vf_coef = 0.2
clip_coef = 0.1
gamma = 0.975
batch_size = 64
max_cycles = 200
total_episodes = 4000
PPO_STEPS = 3

reminder = '''DONT FORGET TO ADD CODE TO SAVE CHECKPOINTS IF YOU WANT TO DO THAT'''
ckpt_name= "GROUP_POLICY_BOTH_INDIVRWD" #"GROUP_POLICY_BOTH_INDIV_RWD_ROUND2bis"
SAVE_PRED_POL = True
SAVE_HIDER_POL = True
print(reminder)

##################################################################################################


Using WALLS
DONT FORGET TO ADD CODE TO SAVE CHECKPOINTS IF YOU WANT TO DO THAT


In [17]:
class Super_Agent(nn.Module):
    #Common agent class for all hiders/seekers
    
    def __init__(self, num_actions, num_agents):
        super().__init__()

        # CNN architecture inspired by DQN for Atari
        self.network = nn.Sequential(
            nn.Conv2d(NUM_THINGS, 32, kernel_size=3, stride=1, padding=1),  # Output: 32 x 7 x 7
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  # Output: 64 x 7 x 7
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),  # Output: 64 x 7 x 7
            nn.ReLU(),
            nn.Flatten(),  # Output: 64 * 7 * 7 = 3136
        )
        self.actor = self._layer_init(nn.Linear(64*GRID_SIZE**2, num_actions), std=0.01) #TODO depends on GRID_SIZE
        self.critic = self._layer_init(nn.Linear(64*GRID_SIZE**2, 1))

    def _layer_init(self, layer, std=np.sqrt(2), bias_const=0.0):
        torch.nn.init.orthogonal_(layer.weight, std)
        torch.nn.init.constant_(layer.bias, bias_const)
        return layer

    def get_value(self, x):
        return self.critic(self.network(x / 1.0))  # Normalize input to [0, 1]

    def get_action_and_value(self, x, action=None):
        hidden = self.network(x / 1.0)  # Normalize input to [0, 1]
        
        logits = self.actor(hidden)
        probs = Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy(), self.critic(hidden)

    

def batchify_obs(obs, device):
    """Converts PZ style observations to batch of torch arrays."""
    # convert to list of np arrays
    obs = np.stack([obs[a] for a in obs], axis=0)
    # convert to torch
    obs = torch.tensor(obs).to(device)

    return obs


def batchify(x, device):
    """Converts PZ style returns to batch of torch arrays."""
    # convert to list of np arrays
    x = np.stack([x[a] for a in x], axis=0)
    # convert to torch
    x = torch.tensor(x).to(device)

    return x


def unbatchify(x, env):
    """Converts np array to PZ style arguments."""
    x = x.cpu().numpy()
    x = {a: x[i] for i, a in enumerate(env.possible_agents)}

    return x

def reshape_obs(observations, env):
    modified_observations = {}
    for self_name, obs in observations.items():
        self_layer = env.agent_layers[self_name]
        enemy_layers = []
        for name, layer_idx in env.agent_layers.items():
            
            if name == self_name:
                self_layer = obs[layer_idx]
            elif name.startswith(self_name[:4]): #starts with the same 4 letters: pred or hide
                friend_layer = obs[layer_idx]
            else:
                enemy_layers.append(obs[layer_idx])
         
        new_obs = [obs[0]] #walls
        new_obs.append(self_layer) #self
        new_obs.append(friend_layer) #friend
        new_obs.append(sum(enemy_layers)) #enemies
        new_obs.append(obs[-1]) #movable walls

        modified_observations[self_name] = np.stack(new_obs, axis = 0)
        
    return modified_observations
        


In [ ]:
wandb.init(
        project="multi-agent-ppo",  # Set your project name
        name=ckpt_name,
        config={
            "env": envname,
            "GRID_SIZE": GRID_SIZE,
            "NUM_THINGS": NUM_THINGS,
            "INITIALIZATIONS": INITIALIZATIONS,
            "IS_TRAINING": IS_TRAINING,
            "ent_coef": ent_coef,
            "vf_coef": vf_coef,
            "clip_coef": clip_coef,
            "gamma": gamma,
            "batch_size": batch_size,
            "max_cycles": max_cycles,
            "total_episodes": total_episodes,
            "PPO_STEPS": PPO_STEPS,
        }
)
"""ALGO PARAMS"""
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lr = 0.0001


""" ENV SETUP """
env = CUSTOMENV

num_agents = len(env.possible_agents)
num_actions = env.action_space(env.possible_agents[0]).n
observation_size = env.observation_space(env.possible_agents[0]).shape

""" LEARNER SETUP """
# Create a list of agents, one for each training agent
training_agents = []
optimizers = []
training_policy_names = [team for team, training in IS_POLICY_TRAINING.items() if training]
training_agent_indices = [i for i, training in enumerate(IS_TRAINING) if training]
frozen_agent_indices = [i for i, training in enumerate(IS_TRAINING) if not training and INITIALIZATIONS[i] != RANDOM]

teams = {"preds":[0,1],"hiders":[2,3]}

pred_policy = Super_Agent(num_actions, num_agents).to(device)
pred_optimizer = optim.Adam(pred_policy.parameters(), lr=lr, eps=1e-5)
if INITIALIZATIONS[0] != RANDOM:
    pred_policy.load_state_dict(torch.load(INITIALIZATIONS[0]))
    print(f'loaded from {INITIALIZATIONS[0]}')
    
hider_policy = Super_Agent(num_actions, num_agents).to(device)
hider_optimizer = optim.Adam(hider_policy.parameters(), lr=lr, eps=1e-5)
if INITIALIZATIONS[2] != RANDOM:
    hider_policy.load_state_dict(torch.load(INITIALIZATIONS[1]))
    print(f'loaded from {INITIALIZATIONS[1]}')

team_policies = {"preds":pred_policy, "hiders":hider_policy}
team_optimizers = {"preds":pred_optimizer, "hiders":hider_optimizer}

# a frozen is one that is NOT TRAINING and NOT RANDOM
for idx in training_agent_indices:
    if idx in teams["hiders"]:
        agent = hider_policy
        optimizer = hider_optimizer
    elif idx in teams["preds"]:
        agent = pred_policy
        optimizer = pred_optimizer

    training_agents.append(agent)
    optimizers.append(optimizer)

frozen_agents = [] # These agents are not random, but are NOT TRAINING ; initialized with a checkpoint
for idx, init in enumerate(INITIALIZATIONS):
    if init != RANDOM and not IS_TRAINING[idx]:
        agent = Super_Agent(num_actions=num_actions, num_agents=num_agents).to(device)
        agent.load_state_dict(torch.load(init))
        agent.eval()
        #freeze weights
        for param in agent.parameters():
            param.requires_grad = False
        print(f' loaded from {init}')
        frozen_agents.append(agent)

""" ALGO LOGIC: EPISODE STORAGE"""
end_step = 0
total_episodic_return = 0
rb_obs = torch.zeros((max_cycles, num_agents, NUM_THINGS,GRID_SIZE,GRID_SIZE)).to(device)
rb_actions = torch.zeros((max_cycles, num_agents)).to(device)
rb_logprobs = torch.zeros((max_cycles, num_agents)).to(device)
rb_rewards = torch.zeros((max_cycles, num_agents)).to(device)
rb_terms = torch.zeros((max_cycles, num_agents)).to(device)
rb_values = torch.zeros((max_cycles, num_agents)).to(device)

""" TRAINING LOGIC """
# Track returns for all agents
all_returns = [[] for _ in range(num_agents)]

for episode in range(1,total_episodes+1):
    # collect an episode
    with torch.no_grad():
        # collect observations and convert to batch of torch tensors
        next_obs, info = env.reset(seed=None)
        # reset the episodic return
        total_episodic_return = 0

        # each episode has num_steps
        for step in range(0, max_cycles):
            #modify observation to get self, friends, enemy position layers
            obs = reshape_obs(next_obs, env)
            
            # rollover the observation
            obs = batchify_obs(obs, device)

            # get action for first agent from the trained agents
            # get random actions for other agents
            actions = torch.zeros(num_agents, dtype=torch.long).to(device)
            logprobs = torch.zeros(num_agents).to(device)
            values = torch.zeros(num_agents).to(device)

            # Process each agent
            for i in range(num_agents):
                if IS_TRAINING[i]:
                    # Find the index of this training agent among training agents
                    train_idx = training_agent_indices.index(i)
                    # Get action and value for training agent
                    agent_obs = obs[i].unsqueeze(0)
                    actions[i], logprobs[i], _, values[i] = training_agents[train_idx].get_action_and_value(agent_obs)
                elif INITIALIZATIONS[i] != RANDOM:
                    #this is a frozen agent (not training, but not random because it has a checkpoint)
                    frozen_idx = frozen_agent_indices.index(i)
                    agent_obs = obs[i].unsqueeze(0)
                    actions[i], logprobs[i], _, values[i] = frozen_agents[frozen_idx].get_action_and_value(agent_obs)

                    logprobs[i] = torch.log(torch.tensor(1.0/num_actions))
                    values[i] = 0.0  # No value estimation for frozen agents
                else:
                    # Random action for random agents
                    actions[i] = torch.randint(0, num_actions, (1,)).to(device)
                    logprobs[i] = torch.log(torch.tensor(1.0/num_actions))
                    values[i] = 0.0  # No value estimation for random agents

            # execute the environment and log data
            next_obs, rewards, terms, truncs, infos = env.step(
                unbatchify(actions, env)
            )

            # add to episode storage
            rb_obs[step] = obs
            rb_rewards[step] = batchify(rewards, device)
            rb_terms[step] = batchify(terms, device)
            rb_actions[step] = actions
            rb_logprobs[step] = logprobs
            rb_values[step] = values

            # compute episodic return
            total_episodic_return += rb_rewards[step].cpu().numpy()

            # if we reach termination or truncation, end
            if any([terms[a] for a in terms]) or any([truncs[a] for a in truncs]):
                end_step = step
                break

    # Train only the specified policies
    for team_train_idx,team_name in enumerate(training_policy_names):
        b_obs = torch.zeros(len(teams[team_name])*end_step, *rb_obs.shape[2:]).to(device)
        b_logprobs = torch.zeros(len(teams[team_name])*end_step, *rb_logprobs.shape[2:]).to(device)
        b_actions = torch.zeros(len(teams[team_name])*end_step, *rb_actions.shape[2:]).to(device)
        b_returns = torch.zeros(len(teams[team_name])*end_step, *rb_returns.shape[2:]).to(device)
        b_values = torch.zeros(len(teams[team_name])*end_step, *rb_values.shape[2:]).to(device)
        b_advantages = torch.zeros(len(teams[team_name])*end_step, *rb_advantages.shape[2:]).to(device)
        
        #Create a big batch containing the data from all the agents in the team
        for train_idx, agent_idx in enumerate(teams[team_name]):
            # Bootstrap value and advantages only for the training agent
            with torch.no_grad():
                rb_advantages = torch.zeros_like(rb_rewards).to(device)
                for t in reversed(range(end_step)):
                    delta = (
                        rb_rewards[t, agent_idx]  # Only specific agent's reward
                        + gamma * rb_values[t + 1, agent_idx] * rb_terms[t + 1, agent_idx]
                        - rb_values[t, agent_idx]
                    )
                    rb_advantages[t, agent_idx] = delta + gamma * gamma * rb_advantages[t + 1, agent_idx]
                rb_returns = rb_advantages + rb_values

            # convert our episodes to batch of individual transitions (only for specific agent)
            b_obs[train_idx*end_step:(train_idx+1)*end_step] = rb_obs[:end_step, agent_idx]
            b_logprobs[train_idx*end_step:(train_idx+1)*end_step] = rb_logprobs[:end_step, agent_idx]
            b_actions[train_idx*end_step:(train_idx+1)*end_step] = rb_actions[:end_step, agent_idx]
            b_returns[train_idx*end_step:(train_idx+1)*end_step] = rb_returns[:end_step, agent_idx]
            b_values[train_idx*end_step:(train_idx+1)*end_step] = rb_values[:end_step, agent_idx]
            b_advantages[train_idx*end_step:(train_idx+1)*end_step] = rb_advantages[:end_step, agent_idx]

        # Optimizing the policy and value network
        b_index = np.arange(len(b_obs))
        clip_fracs = []
        for repeat in range(PPO_STEPS):
            # shuffle the indices we use to access the data
            np.random.shuffle(b_index)
            for start in range(0, len(b_obs), batch_size):
                # select the indices we want to train on
                end = start + batch_size
                batch_index = b_index[start:end]

                _, newlogprob, entropy, value = team_policies[team_name].get_action_and_value(
                    b_obs[batch_index], b_actions.long()[batch_index]
                )
                logratio = newlogprob - b_logprobs[batch_index]
                ratio = logratio.exp()

                with torch.no_grad():
                    # calculate approx_kl http://joschu.net/blog/kl-approx.html
                    old_approx_kl = (-logratio).mean()
                    approx_kl = ((ratio - 1) - logratio).mean()
                    clip_fracs += [
                        ((ratio - 1.0).abs() > clip_coef).float().mean().item()
                    ]

                # normalize advantages
                advantages = b_advantages[batch_index]
                advantages = (advantages - advantages.mean()) / (
                    advantages.std() + 1e-8
                )

                # Policy loss
                pg_loss1 = -b_advantages[batch_index] * ratio
                pg_loss2 = -b_advantages[batch_index] * torch.clamp(
                    ratio, 1 - clip_coef, 1 + clip_coef
                )
                pg_loss = torch.max(pg_loss1, pg_loss2).mean()

                # Value loss
                value = value.flatten()
                v_loss_unclipped = (value - b_returns[batch_index]) ** 2
                v_clipped = b_values[batch_index] + torch.clamp(
                    value - b_values[batch_index],
                    -clip_coef,
                    clip_coef,
                )
                v_loss_clipped = (v_clipped - b_returns[batch_index]) ** 2
                v_loss_max = torch.max(v_loss_unclipped, v_loss_clipped)
                v_loss = 0.5 * v_loss_max.mean()

                entropy_loss = entropy.mean()
                loss = pg_loss - ent_coef * entropy_loss + v_loss * vf_coef

                team_optimizers[team_name].zero_grad()
                loss.backward()
                team_optimizers[team_name].step()

        # Store returns for the training agents
        y_pred, y_true = b_values.cpu().numpy(), b_returns.cpu().numpy()
        var_y = np.var(y_true)
        explained_var = np.nan if var_y == 0 else 1 - np.var(y_true - y_pred) / var_y

    # Accumulate returns for all agents
    for i in range(num_agents):
        all_returns[i].append(total_episodic_return[i])

    if episode % 10 == 0:
        print(f"Training episode {episode}")
        print(f"Episodic Return: {(total_episodic_return)}")

        # Print smoothed returns for each agent
        for i in range(num_agents):
            status = None

            if IS_TRAINING[i]:
                status = "Training"
            elif INITIALIZATIONS[i] == RANDOM:
                status = "Random"
            else:
                status = "Frozen"
                
            print(f"Smoothed Returns for agent_{i} ({status}): {np.mean(all_returns[i][-20:])}")

        print(f"Episode Length: {end_step}")
        print("")
        print(f"Value Loss: {v_loss.item()}")
        print(f"Policy Loss: {pg_loss.item()}")
        print(f"Old Approx KL: {old_approx_kl.item()}")
        print(f"Approx KL: {approx_kl.item()}")
        print(f"Clip Fraction: {np.mean(clip_fracs)}")
        print(f"Explained Variance: {explained_var.item()}")
        print("\n-------------------------------------------\n")

    #log all with wandb
    wandb.log({
        "Ep return pred1": total_episodic_return[0],
        "Ep return pred2": total_episodic_return[1],
        "Ep return hider1": total_episodic_return[2],
        "Ep return hider2": total_episodic_return[3],
        "Episode Length": end_step,
        "Value Loss": v_loss.item(),
        "Policy Loss": pg_loss.item(),
        "Old Approx KL": old_approx_kl.item(),
        "Approx KL": approx_kl.item(),
        "Clip Fraction": np.mean(clip_fracs),
        "Explained Variance": explained_var.item()
    })

    #if for pred_1 (index 0) episode return and smoothed are greater than -200, save the model

    # if total_episodic_return[0] > -210 and np.mean(all_returns[0][-20:]) > -210:
    #     #create dir
    #     import os
    #     if not os.path.exists('./models'):
    #         os.makedirs('./models')f
    #     #save just state dict for 0
    #     torch.save(agents[0].state_dict(), f'./models/agentwalls_{episode}.ckpt')
    #     exit(1)



    #if reward greater than 600 for hider_1 both for last and smoothed for last 5
    #every 100 epochs save the 2 models

    if episode % 100 == 0:
        if SAVE_PRED_POL:
            torch.save(pred_policy.state_dict(), f'./models/PRED_{ckpt_name}_{episode}.ckpt')
        if SAVE_HIDER_POL:
            torch.save(hider_policy.state_dict(), f'./models/HIDER_{ckpt_name}_{episode}.ckpt')

Approx KL,▂▁▂▆▂▂▄▁▄▂▁▁▂▃▂▂▃▃▄▁▁▁▂▁▃▄▂▁▃▄▁▇▂▃▂█▂▄▄▁
Clip Fraction,▂▃▃▄▂▃▃▄▄▁▅▁▂▃▃▁▃▄█▂▃▂▂▂▂▂▃▄▃▄▂▂▃▂▃▂▃▄▃▃
Ep return hider1,█▇██▆▇▆▆▄█▇█▅▄▆▅▆▅▄▅▄▃▃▅▄▁▂▄▃▄▄▄▂▃▃▃▅▃▄▃
Ep return hider2,▄▅▆▆▇▇█▅▇▅▅██▆▇▇▇▇▅▅▃▃▅▄▅▁▆▂▃▁▂▅▃▄▃▂▂▄▄▂
Ep return pred1,▃▂▃▃▃▃▁▂▄▄▂▃▃▂▃▄▃▄▇▄▅▇▆▅▅▅▆▆▆▅▆█▅▇▇██▇▇▆
Ep return pred2,▂▄▂▅▃▄▄▅▃▂▁▂▅▂▂▄▅▄▇▄▅▄▆▇▇▆▅█▆▇▇▆█▇▇▇▇▆▆▇
Episode Length,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Explained Variance,▆▇▆▆▅█▁▃▆▇▇▆▅▆▆▇▄▅▆▅▅▅▅▅█▆▆▇▆▆▆▅▇▇▅▇█▆▇█
Old Approx KL,▃▄▄▄▄▂▂▃▅▅▁▄▃▃▄▁▃▃▄▃▃▃▅▂▆▂▃▅▆▂▁▄▄▃▄▆▅▃█▄
Policy Loss,▇▁▃▆▂▇▇▃▆▇▃▅▄▄▄▅▄▃▇▂▇▅▅▃▄▄▅▆▅▄▅▆▅▅▅▄▂▃█▄
Value Loss,█▂▁▃▃▂▁▅▇▃▃▁▁▁▁▅▁▆▃▃▂▂▄▃▂▃▃▃▃▂▂▂▃▂▃▂▁▂▄▂


Training episode 10
Episodic Return: [ 1254.8323   867.0374 -1296.9248 -1010.4286]
Smoothed Returns for agent_0 (Training): 105.19563293457031
Smoothed Returns for agent_1 (Training): 431.97479248046875
Smoothed Returns for agent_2 (Training): -303.53289794921875
Smoothed Returns for agent_3 (Training): -374.6353454589844
Episode Length: 199

Value Loss: 16306.029296875
Policy Loss: 97.17435455322266
Old Approx KL: -0.020823180675506592
Approx KL: 0.0020114779472351074
Clip Fraction: 0.03178146339598156
Explained Variance: 5.1081180572509766e-05

-------------------------------------------

Training episode 20
Episodic Return: [ 1633.4447   -177.20023  -385.97424 -1264.9896 ]
Smoothed Returns for agent_0 (Training): 156.82447814941406
Smoothed Returns for agent_1 (Training): 620.79345703125
Smoothed Returns for agent_2 (Training): -472.3787536621094
Smoothed Returns for agent_3 (Training): -500.73358154296875
Episode Length: 199

Value Loss: 3351.7294921875
Policy Loss: 24.105815887451